In [17]:
%run '/home/lena/Desktop/Application Project/src/ibm1.py'
%run '/home/lena/Desktop/Application Project/src/ibm2.py'
%run '/home/lena/Desktop/Application Project/src/pbmt.py'
%run '/home/lena/Desktop/Application Project/src/utils.py'
%run '/home/lena/Desktop/Application Project/src/test.py'
import numpy as np

In [ ]:
import math
TEST = False 
nr_used_sentences = 50 #1800 for 1model,25steps  # of 688670 lines
path = "./src/Paralel Corpus/"
e, f = read_in_corpus(nr_used_sentences, path)

training_split = 0.9
e_train, e_test = split_dataset(e, training_split)
f_train, f_test = split_dataset(f, training_split)

if TEST:
    # we don't use real data but instead [['house','the','the'],['the','the','book'],['a','book']] etc
    e_train, f_train, all_alignments_train, e_test, f_test, all_alignments_test = initialize_test_sets()

max_steps = 75#25

t_e2f_ibm1 = EM_IBM_Model_1(e_train, f_train, max_steps)
t_e2f, a_e2f = EM_IBM_Model_2(e_train, f_train, t_e2f_ibm1, max_steps)
t_f2e_ibm1 = EM_IBM_Model_1(f_train, e_train, max_steps)
t_f2e, a_f2e = EM_IBM_Model_2(f_train, e_train, t_f2e_ibm1, max_steps)

if TEST:
    # alignment probabilities a gives different results than implementation in the nltk library
    #compare_ibm_1_nltk(t_e2f_ibm1, max_steps, e_train, f_train)
    #compare_ibm_2_nltk(t_e2f, max_steps, a_e2f, e_train, f_train)
    #compare_ibm_2_nltk(t_f2e, max_steps, a_f2e, f_train, e_train)
    
    # compare a's train() and EM_IBM_Model_2() > 99% same a
    #compare_a_ibm_2_train(t_e2f, max_steps, a_e2f, e_train, f_train)
    #compare_a_ibm_2_train(t_f2e, max_steps, a_f2e, f_train, e_train)

    # calculate sum of a over all words f: should be 1 
    #test_sum_a_is_one(a_e2f, f_train)
    #test_sum_a_is_one(a_f2e, e_train)
    #compare_a_nltk_train(t_e2f_ibm1, max_steps, e_train, f_train)

    
"""# Testing
# only works for test test_set, not for real data
correct1 = 0
correct2 = 0
bool_test = [True, True, True, False, False, False] #correct prob values around[1, 0.7, 0.9, 0.1, 0.01, 0]
for i in range(0, len(f_test)):
    f = f_test[i]
    e = e_test[i]
    p1 = prob_e_given_f_1(e, f, 1, t_e2f_ibm1)
    if (p1 > 0.5) == bool_test[i]: correct1 += 1
    print('p_IBM1(', e, '|', f, ') =', p1)
    p2 = prob_e_given_f_2(e, f, 1, t_e2f, a_e2f)
    if (p2 > 0.5) == bool_test[i]: correct2 += 1
    print('p_IBM2(', e, '|', f, ') =', p2)
print('IBM Model 1 :', int(100*(correct1/len(f_test))),'% correct')
print('IBM Model 2 :', int(100*(correct2/len(f_test))),'% correct')
"""

In [ ]:
# Phrase - based MT
#Input: sets of sentences e and f, t- and a-table in both directions
#Output: alignments for sentences in sets
def word_alignment(e, f, t_e2f, a_e2f, t_f2e, a_f2e):
    all_a = {}
    for si in range(0, len(f)):
        e2f = viterbi_alignment(e[si], f[si], t_e2f, a_e2f)
        f2e = viterbi_alignment(f[si], e[si], t_f2e, a_f2e)
        a = combine(f2e, e2f)
        if (a == all_alignments_train[si]): print('Right alignment :)\n')
        else: 
            print(e[si],f[si],'\ne2f    ',e2f,'\nf2e    ',f2e,'\ncombine', a)
            print('Returned wrong alignment',a,'\nRight alignment would be',all_alignments_train[si],'\n')
        all_a[si] = a
    return all_a

# IBM Model 2 for word alignment
all_a_train = word_alignment(e_train, f_train, t_e2f, a_e2f, t_f2e, a_f2e)
#all_a_test = word_alignment(e_test, f_test, t_e2f, a_e2f, t_f2e, a_f2e)

In [ ]:
#HEREEEEE
"""
{['house']:{['Haus']:1}, (['house', 'the'], ['das', 'Haus']), (['house', 'the', 'the'], ['das', 'Haus']), (['the'], ['das']), (['the', 'the'], ['das']), (['the'], ['das', 'Buch']), (['the', 'the'], ['das', 'Buch']), (['the', 'the', 'book'], ['das', 'Buch']), (['a'], ['ein']), (['a', 'book'], ['ein', 'Buch']), (['book'], ['Buch']), (['house'], ['Haus'])]
"""
e_train = [['michael','assumes','that','he','will','stay','in','the','house']]
f_train = [['michael','geht','davon','aus',',','dass','er','im','haus','bleibt']]
all_a_train = {0:{0:[0],1:[1,2,3],2:[5],3:[6],4:[9],5:[9],6:[7],7:[7],8:[8]}}
# all_a[a][b] alignment for b word in e for sentence a
#"""
counts = phrase_extraction(e_train, f_train, all_a_train, 3)
#counts = phrase_extraction(e_test, f_test, all_a_test, 3)
print('counts:',counts,'\n')

#phrase translation table
print('PT_prob([\'house\'], [\'Haus\']) =', PT_prob(['house'], ['Haus'], counts))
print('PT_prob([\'the\',\'house\'], [\'das\',\'Haus\']) =', PT_prob(['the','house'],['das','Haus'],counts),'\n')

#bigram language model
unigram_counts, bigram_counts = count_grams(e_test)
#print('unigram_counts',unigram_counts,'\n bigram_counts', bigram_counts,'\n')

prev = 'book'
cur = 'a'
print('P(',cur,'|',prev,') should be high',LM_prob(prev, cur, unigram_counts, bigram_counts))
cur = 'house'
print('P(',cur,'|',prev,') should be low', LM_prob(prev, cur, unigram_counts, bigram_counts))

# TODO phrase-based statistical machine translation model
# • the phrase translation table φ(f|e);
# • the reordering model d;
# • the language model pLM(e).
# e_best = argmax_e prod_i φ(f|e) * d(start_i - end_i_min_1 - 1) * prod_i PLM(e_i|e_i_min_1)
for sentence in range(0, len(all_alignments_test)):
    f = f_test[sentence]
    e = e_test[sentence]
    alignment = all_alignments_test[sentence]
    prob = prob_e_given_f(e, f, alignment, counts, unigram_counts, bigram_counts)

In [ ]:
#TODO unaligned cases!
#model null alignments, some word doesnt have a word to align to 
#> just align it with the highest probable one

#debug ibm 2
#IBM Model 2: How to calculate a correctly?
#my a is 1 when it should be low :(
#implementation exactly like in book
#eg sum is supposed to always be 1
#use ibm model 2 in slides
#compare subresults if they are right, count, stotal etc
#look into indexing, starting 0 / 1?

# TODO translation probability is not working for phrases containing several words
#p(e|f) for IBM Model 2: Gives result 4 should be 1?
#is sum wrong maybe?
#> compare with other implementations

#calculating p(e|f) phrase based model

#decoding > example target sentences - do we even need the test set?
#&gen alignments or manual?

#TODO save t, a in files & load them?

#structure code more? classes ML model, so it's intuitive to execute

#remove special chars preprocessing
#Application: Preprocessing convert Turkish characters!
#Remove non-ASCII characters
#lowercase
#multi-character whitespaces to a single whitespace character
#Remove URLs, numbers, leading and trailing whitespaces

#efficient n-gram with trie
#only store ngrams above a certain count threshold eg 4 saving 90%
# save a significant amount of memory if we only store terms with some minimum count.

#calc how long stuff takes
#look into efficiency issues, use hashing?
# TODO For the estimation of the phrase translation probabilities, not all
# phrase pairs have to be loaded into memory. It is possible to efficiently
# estimate the probability distribution by storing and sorting the extracted
# phrases on disk. Similarly, when using the translation table for the translation of a single sentence, only a small fraction of it is needed and may
# be loaded on demand.
# think about saving stuff in file
# • Phrase translation table typically bigger than corpus
# ... even with limits on phrase lengths (e.g., max 7 words)
# → Too big to store in memory?
# • Solution for training
# – extract to disk, sort, construct for one source phrase at a time
# • Solutions for decoding
# – on-disk data structures with index for quick look-ups
# – suffix arrays to create phrase pairs on demand
# TODO use numpy for everything
# maybe other data structure ['the','house']:[['das','Haus'],['ein','Haus']]
# faster sometimes but slower if translation in other direction

In [2]:
####### IBM MODELS #######

# read in data
nr_used_sentences = 1800
path = "./src/Paralel Corpus/"
e, f = read_in_corpus(nr_used_sentences, path)

training_split = 0.9
e_train, e_test = split_dataset(e, training_split)
f_train, f_test = split_dataset(f, training_split)
max_steps = 30

#TODO remove
e_train += [['the' ,'man', 'understood','this','extinguished','his','cigarette','went','to','the','hallway','after','shaking','the','womans','hand']]
f_train += [['adam', 'bunu', 'anladı', 'sigarasını', 'söndürdü', 'kadının', 'elini', 'sıkarak', 'hole', 'çıktı']]

# execute IBM Model 1 and 2 each two times (English-Turkish and Turkish-English)
t_e2f_ibm1 = EM_IBM_Model_1(e_train, f_train, max_steps)
t_e2f, a_e2f = EM_IBM_Model_2(e_train, f_train, t_e2f_ibm1, max_steps)
t_f2e_ibm1 = EM_IBM_Model_1(f_train, e_train, max_steps)
t_f2e, a_f2e = EM_IBM_Model_2(f_train, e_train, t_f2e_ibm1, max_steps)

# testing
examples_f = [['adam', 'bunu', 'anladı', 'sigarasını', 'söndürdü', 'kadının', 'elini', 'sıkarak', 'hole', 'çıktı'],['girdiği', 'bütün', 'işlerde', 'bir', 'terslikle', 'karşılaşmış', 'şimdi', 'de', 'demiryolu', 'işletmesine', 'geçmişti'], ['duruşma', 'aralarında', 'çayım', 'içip', 'sigarasını', 'tüttürerek', 'biraz', 'siyasetten', 'biraz', 'günlük', 'işlerden', 'biraz', 'kâğıt', 'oyunlarından', 'daha', 'çok', 'da', 'atamalardan', 'söz', 'açtığı', 'olurdu']]
examples_e = [{0: ['the' ,'man', 'understood','this','extinguished','his','cigarette','went','to','the','hallway','after','shaking','the','womans','hand'], 1: ['the' ,'man', 'understood','this','extinguished','his','cigarette','went','to','the','hall','while','shaking','hands','with','the','woman'], 2: ['the' ,'man', 'got','it','and','put','the','cigarette','off','went','to','the','hallway','shaking','the','hand','of','the','woman']},\
            {0:['in','every','work','he','tried','he','struggled','and','was','working','in','the','railway','department','now'],1:['in','every','work','he','entered','he','came','across','a','struggle','so','now','he','switched','into','the','railroad','business'], 2:['in','every','job','he','tried','he','was','misfortunate','and','now','he','switched','to','working','in','the','railroad','business'], 3:['he', 'had', 'ruined', 'his', 'prospects', 'in', 'a', 'number', 'of', 'positions', 'and', 'was', 'now', 'serving', 'in', 'the', 'railway', 'department']},\
            {0:['Between','different','trials','he','would','drink','his','tea','smoke','his','cigarette','and','talk','about','some','politics','daily','errands','some','cardgames','and','mostly','about','appointments'], 1:['in','the', 'intervals', 'between', 'the', 'sessions', 'he', 'smoked', 'drank', 'tea', 'chatted', 'a', 'little', 'about', 'politics','a', 'little', 'about', 'general', 'topics', 'a', 'little' ,'about','cards', 'but', 'most', 'of', 'all', 'about', 'official', 'appointments']}]


start training IBM Model 1
step 25 of 30
IBM Model 1 training finished.
start training IBM Model 2
step 25 of 30
IBM Model 2 training finished.
start training IBM Model 1
step 25 of 30
IBM Model 1 training finished.
start training IBM Model 2
step 25 of 30
IBM Model 2 training finished.
IBM Model 1 :p( ['the', 'man', 'understood', 'this', 'extinguished', 'his', 'cigarette', 'went', 'to', 'the', 'hallway', 'after', 'shaking', 'the', 'womans', 'hand'] | ['adam', 'bunu', 'anladı', 'sigarasını', 'söndürdü', 'kadının', 'elini', 'sıkarak', 'hole', 'çıktı'] ) = 2.4423984532665256e-20
IBM Model 2: p( ['the', 'man', 'understood', 'this', 'extinguished', 'his', 'cigarette', 'went', 'to', 'the', 'hallway', 'after', 'shaking', 'the', 'womans', 'hand'] | ['adam', 'bunu', 'anladı', 'sigarasını', 'söndürdü', 'kadının', 'elini', 'sıkarak', 'hole', 'çıktı'] ) = 6.721854537670273e-13
Some contained words were not in the training set.
IBM Model 1 :p( ['the', 'man', 'understood', 'this', 'extinguished', '

In [18]:
for i in range(0, len(examples_f)):
    f = examples_f[i]
    for e in examples_e[i].values():
        p1 = prob_e_given_f_1(e, f, 1e19, t_e2f_ibm1)
        print('IBM Model 1 :p(', e, '|', f, ') =', p1)
        p2 = prob_e_given_f_2(e, f, 1e12, t_e2f, a_e2f)
        print('IBM Model 2: p(', e, '|', f, ') =', p2)

IBM Model 1 :p( ['the', 'man', 'understood', 'this', 'extinguished', 'his', 'cigarette', 'went', 'to', 'the', 'hallway', 'after', 'shaking', 'the', 'womans', 'hand'] | ['adam', 'bunu', 'anladı', 'sigarasını', 'söndürdü', 'kadının', 'elini', 'sıkarak', 'hole', 'çıktı'] ) = 0.24423984532665258
IBM Model 2: p( ['the', 'man', 'understood', 'this', 'extinguished', 'his', 'cigarette', 'went', 'to', 'the', 'hallway', 'after', 'shaking', 'the', 'womans', 'hand'] | ['adam', 'bunu', 'anladı', 'sigarasını', 'söndürdü', 'kadının', 'elini', 'sıkarak', 'hole', 'çıktı'] ) = 0.6721854537670273
Some contained words were not in the training set.
IBM Model 1 :p( ['the', 'man', 'understood', 'this', 'extinguished', 'his', 'cigarette', 'went', 'to', 'the', 'hall', 'while', 'shaking', 'hands', 'with', 'the', 'woman'] | ['adam', 'bunu', 'anladı', 'sigarasını', 'söndürdü', 'kadının', 'elini', 'sıkarak', 'hole', 'çıktı'] ) = 3.1559376057768554e-21
Some contained words were not in the training set.
IBM Model 2:

In [16]:
####### PHRASE BASED MODEL #######
# alignment e:[f]
examples_a = [{0: {0:[0], 1:[0], 2:[2], 3:[1], 4:[4], 5:[3], 6:[3], 7:[9], 8:[8], 9:[8], 10:[8], 11:[7], 12:[7], 13:[5], 14:[5],15:[6]}, 1: {0:[0], 1:[0], 2:[2], 3:[1], 4:[4], 5:[3], 6:[3], 7:[9], 8:[8], 9:[8], 10:[8], 11:[7], 12:[7], 13:[6], 14:[5],15:[5],16:[5]}, 2: {0:[0], 1:[0], 2:[2], 3:[1], 4:[4], 5:[4], 6:[3], 7:[3], 8:[4], 9:[9], 10:[8], 11:[8], 12:[8], 13:[7], 14:[6],15:[6],16:[5],17:[5],18:[5]}},\
{0: {0:[2], 1:[1], 2:[2], 3:[0], 4:[0], 5:[3,4,5], 6:[3,4,5], 7:[7], 8:[10], 9:[10], 10:[9], 11:[9], 12:[8], 13:[9], 14:[6]}, 1: {0:[2], 1:[1], 2:[2], 3:[0], 4:[0], 5:[5], 6:[5], 7:[5], 8:[4], 9:[4], 10:[7], 11:[6], 12:[10], 13:[10], 14:[9],15:[9],16:[8],17:[9]}, 2: {0:[2], 1:[1], 2:[2], 3:[0], 4:[0], 5:[3,4,5], 6:[3,4,5], 7:[3,4,5], 8:[7], 9:[6], 10:[10], 11:[10], 12:[10], 13:[10], 14:[9],15:[9],16:[8],17:[9]}, 3: {0:[4], 1:[3,4,5], 2:[3,4,5], 3:[3,4,5], 4:[3,4,5], 5:[2], 6:[1], 7:[1], 8:[1], 9:[2], 10:[7], 11:[10], 12:[6], 13:[10], 14:[9],15:[9],16:[8],17:[9]}},\
{0: {0:[1], 1:[0], 2:[0], 3:[3], 4:[3], 5:[3], 6:[2], 7:[2], 8:[5], 9:[4], 10:[4], 11:[18,19,20], 12:[18,19,20], 13:[6], 14:[7],15:[9],16:[10],17:[11],18:[12,13],19:[14],20:[14,15],21:[17],22:[17]}, 1: {0: [1], 1: [1], 2: [1], 3: [0], 4: [0], 5: [0], 6: [4,5], 7: [4,5], 8: [3], 9: [2], 10: [18,19,20], 11: [6],  12: [6], 13: [7], 14: [7], 15: [8], 16: [8], 17: [10], 18: [9], 19: [10], 20: [11], 21: [11], 22: [13],23:[12,13],24:[16],25:[14,15],26:[14,15],27:[14,15],28:[17],29:[17],30:[17]}}]

# use IBM Model 2 for Viterbi alignments & combine them

all_a_train = {}
for si in range(0, len(f_train)):
    e2f = viterbi_alignment(e_train[si], f_train[si], t_e2f, a_e2f)
    f2e = viterbi_alignment(f_train[si], e_train[si], t_f2e, a_f2e)
    a = combine(f2e, e2f)
    all_a_train[si] = a
all_a_train[len(f_train)-1] = examples_a[0][0]
# extract phrases & estimate phrase translation probabilities

max_phrase_len = 8
phrase_counts = phrase_extraction(e_train, f_train, all_a_train, max_phrase_len)

# train bigram language model LM
unigram_counts, bigram_counts = count_grams(e_train)

# testing
# aligned_phrases format [{index: {e_to: (e_from, f_from, f_to)}}
aligned_phrases = [{0: {1:(0,0,0), 2:(2,2,2), 3:(3,1,1), 6:(4,3,4), 10:(7,8,9), 15:(11,5,7)}, 1: {1:(0,0,0), 2:(2,2,2), 3:(3,1,1), 6:(4,3,4), 10:(7,8,9), 16:(11,5,7)}, 2: {1:(0,0,0), 2:(2,2,2), 3:(3,1,1), 8:(4,3,4), 12:(9,8,9), 18:(13,5,7)}}]
for sentence in range(0, 1):#len(examples_f)):#TODO for all sentences & not only first after I filled aligned_phrases
    f = examples_f[sentence]
    for index, e in examples_e[sentence].items():
        a = examples_a[sentence][index]
        phrases_a = aligned_phrases[sentence][index]
        prob = prob_e_given_f(e, f, phrase_counts, unigram_counts, bigram_counts, phrases_a)
        print('p(', e, '|', f, ') =', prob,'\n')

count_grams
calculate PT_prob for ['the', 'man'] ['adam'] = 0.5
d( 0 )= 1.0
calculate PT_prob for ['understood'] ['anladı'] = 0.5
d( 1 )= 0.5
calculate PT_prob for ['this'] ['bunu'] = 0.00641025641025641
d( -2 )= 0.25
calculate PT_prob for ['extinguished', 'his', 'cigarette'] ['sigarasını', 'söndürdü'] = 1.0
d( 1 )= 0.5
calculate PT_prob for ['went', 'to', 'the', 'hallway'] ['hole', 'çıktı'] = 1.0
d( 3 )= 0.125
calculate PT_prob for ['after', 'shaking', 'the', 'womans', 'hand'] ['kadının', 'elini', 'sıkarak'] = 1.0
d( -5 )= 0.03125
LM_prob( the | START )= 0.026390041493775933
LM_prob( man | the )= 0.00047393364928909954
LM_prob( understood | man )= 0.00044974139869574995
LM_prob( this | understood )= 0.00045351473922902497
LM_prob( extinguished | this )= 0.00041675349031048136
LM_prob( his | extinguished )= 0.0004540295119182747
LM_prob( cigarette | his )= 0.0006561679790026247
LM_prob( went | cigarette )= 0.0004538234626730202
LM_prob( to | went )= 0.0006804264005443411
LM_prob( the |